## 1. Library Import

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import random

from tqdm.notebook import tqdm
tqdm.pandas()

import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

from sklearn.model_selection import train_test_split

In [2]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']= '0'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count()) 
print('Current cuda device:', torch.cuda.current_device()) 

Device: cuda
Count of using GPUs: 1
Current cuda device: 0


In [3]:
seed = 42

def set_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed)

## 2. Hyper-Parameter

In [4]:
weight_path = 'beomi/kcbert-base'
new_weight_path = './fp_result/law_further_pretrained'
max_len = 300
epochs = 20
batch_size = 32

## 3. Data Load

In [5]:
law_data = pd.read_csv('./data/law_fp.csv')

del law_data['Unnamed: 0']

In [6]:
law_data

,subdomain,pre_content,content,label
0,민법,그 당시 알 수 있었거나 또는 알고서 이를 주장하지 않았던 사항에 한하여서만 기판력...,그 당시 알 수 있었거나 또는 알고서 이를 주장하지 않았던 사항에 한하여서만 기판력...,0
1,민법,이러한 흐름은 표현의 자유에 대하여 강한 보호를 부여해 온 미국의 전통과 관련이 깊...,이러한 흐름은 표현의 자유에 대하여 강한 보호를 부여해 온 미국의 전통과 관련이 깊...,0
2,민법,양도 제한 조건부 주식은 임직원에게 일정 기간 동안 처분이 금지되는 주식을 지급하는...,양도 제한 조건부 주식은 임직원에게 일정 기간 동안 처분이 금지되는 주식을 지급하는...,0
3,민법,대법원도 판례 과 판례 에서 이와 동일하게 판단하였다,"대법원도 판례 1과 판례 3에서, 이와 동일하게 판단하였다.",0
4,민법,제 조 제 조에서의 해제를 하면 증여 계약은 처음부터 절대적으로 무효가 되고 이러한...,"제555조∼제558조에서의 해제를 하면, 증여 계약은 처음부터 절대적으로 무효가 되...",0
...,...,...,...,...
99995,형법,진입하기 전 자동조타에서 수동 조타로 바꾸었다는 것으로도 짐작 가능하다,진입하기 전 자동조타에서 수동 조타로 바꾸었다는 것으로도 짐작 가능하다.,4
99996,형법,자수범과 의무범의 경우에는 적용될 수 없음을 논의의 전제로서 못박고 있다,자수범과 의무범의 경우에는 적용될 수 없음을 논의의 전제로서 못박고 있다.,4
99997,형법,이를 킨트호이저의 말을 빌어 표현하면 정범은 그 사건을 형법적 의미에서 그 사람의 ...,"이를 킨트호이저의 말을 빌어 표현하면, 정범은 그 사건을 형법적 의미에서 그 사람의...",4
99998,형법,한편 낙태행위의 시점까지 태아가 생존하고 있지 않으면 안되고 이미 사망한 태아는 낙...,"한편 낙태행위의 시점까지 태아가 생존하고 있지 않으면 안되고, 이미 사망한 태아는 ...",4


## 4. Data Split

In [7]:
# train_pretrain / fine_tuning split
train_data, val_data, train_label, val_label = train_test_split(law_data[law_data.columns[:-1]],
                                                                law_data['label'],
                                                                test_size = 0.2,
                                                                random_state = 42)

In [8]:
train_label.value_counts()

4    16022
3    16013
2    15998
1    15985
0    15982
Name: label, dtype: int64

In [9]:
val_label.value_counts()

0    4018
1    4015
2    4002
3    3987
4    3978
Name: label, dtype: int64

In [10]:
law_train = train_data.copy()
law_train['label'] = train_label

law_val = val_data.copy()
law_val['label'] = val_label

In [11]:
print(len(law_train), len(law_val))

80000 20000


In [12]:
law_train.to_csv('./data/law_fp_train.csv')
law_val.to_csv('./data/law_fp_val.csv')

## 5. Vocab 추가

### (1) 기존 vocab 확인

In [13]:
tokenizer = AutoTokenizer.from_pretrained(weight_path)
len(tokenizer)

30000

In [14]:
vocab_tokens = [tok for tok, index in tokenizer.get_vocab().items()]
print(vocab_tokens)

['선택이', '시위를', '터지면', '않도록', '슥', '동조하는', '뎄', '부대', '개천', '##하는거냐', '##옘', '공산독재', '##부름', '핶', '것이라고', '할인', '##입니까', '##스도', '조국에게', '주제를', '뇌가', '##쩔', '룍', '##언', '##쟙', '##중국', '기레기들아', '나이', '싸지르', '👑', '##꺼냐', '장하', '말이죠', '러시아', '##조리', '원칙을', '🌑', '##싱', '어딨냐', '생산', '지지자들', '준비를', '💏', '##강', '##할꼬', '##지금', '없는거', '##할꺼다', '##어놓고', '천안', '않지', '##음에', '##짓만', '##잊', '욕이나', '굻', '비극', '바미당', '##읨', '흔들고', '##퐁', '##대국', '##인한', '전멸', '주둥이', '거기에', '생각없는', '##킵시다', '훻', '받으려고', '류', '##지는', '##산이', '##선택', '이새끼는', '조작된', '선심성', '##일보', '민식이', '##임', '당장', '나가면', '쓰레기들은', '봅', '##같네', '얇', '몇년', '##잔아', '##이드', '##인것처럼', '##여자는', '기본도', '없네', '온나라가', '##훑', '##쥽', '연합', '##수막', '##말든', '청와', '반복', '대권', '##💖', '문어벙', '##😕', '전가', '계기가', '감싸', '고민정이', 'OOO들', '지난번', '양', '토', '##근무', '##국은', '덤벼', '사퇴하세요', '홧', '##끼가', '##읿', '##셨습니다', '##정부는', '수사하라', '할아버', '버닝썬', '있구나', '봐주고', '##하나도', '왜구들', '##나베', '옄', '가로', '##속으로', '이것만', '##짓고', '결과를', '등장', '벝', '##욯', '##회복', '졸라', '

In [15]:
print({index:tok for tok, index in tokenizer.get_vocab().items()})

{29879: '선택이', 25099: '시위를', 18669: '터지면', 17639: '않도록', 1984: '슥', 26059: '동조하는', 855: '뎄', 13266: '부대', 20850: '개천', 11907: '##하는거냐', 5863: '##옘', 25310: '공산독재', 29651: '##부름', 3378: '핶', 23821: '것이라고', 19665: '할인', 9951: '##입니까', 17562: '##스도', 22717: '조국에게', 29170: '주제를', 9829: '뇌가', 4684: '##쩔', 1229: '룍', 4151: '##언', 7283: '##쟙', 12752: '##중국', 22150: '기레기들아', 8803: '나이', 24170: '싸지르', 3734: '👑', 17503: '##꺼냐', 16366: '장하', 21531: '말이죠', 9991: '러시아', 10223: '##조리', 29780: '원칙을', 3571: '🌑', 4097: '##싱', 18696: '어딨냐', 9780: '생산', 12889: '지지자들', 28134: '준비를', 3772: '💏', 4479: '##강', 24077: '##할꼬', 15244: '##지금', 13375: '없는거', 29328: '##할꺼다', 12621: '##어놓고', 18486: '천안', 18007: '않지', 18775: '##음에', 18991: '##짓만', 5566: '##잊', 25840: '욕이나', 357: '굻', 18459: '비극', 22237: '바미당', 7693: '##읨', 22634: '흔들고', 5122: '##퐁', 14147: '##대국', 21103: '##인한', 24830: '전멸', 10683: '주둥이', 10412: '거기에', 18060: '생각없는', 18772: '##킵시다', 3496: '훻', 23082: '받으려고', 1255: '류', 8195: '##지는', 14139: '##산이', 18

### (2) 추가할 단어 선정

In [16]:
law_word = pd.read_excel('./data/word/law_word.xlsx')
law_word_list = list(law_word['word'])

In [17]:
print(len(law_word_list))
print(len(set(law_word_list))) # 중복 체크
print(law_word_list)

10000
10000
['있다', '경우', '규정', '이러한', '문제', '필요', '인정', '행위', '관련', '적용', '것으로', '때문', '또는', '하는', '것이', '관한', '판단', '발생', '해당', '경우에는', '법률', '같은', '계약', '위한', '이를', '의미', '회사', '기업', '책임', '사건', '제도', '의무', '관계', '이에', '결과', '보호', '해석', '결정', '대해서', '권리', '내용', '민법', '원칙', '개정', '대해', '국가', '고려', '없는', '같이', '기준', '정보', '때문에', '범죄', '거래', '판결', '여부', '이익', '존재', '판례', '우리', '포함', '이유', '근거', '주주', '주장', '조세', '있어서', '형법', '개념', '논의', '가능', '청구', '통해', '제한', '필요가', '사용', '법적', '과세', '목적', '헌법', '다음', '행사', '중요', '위해', '방법', '범위', '법인', '미국', '대상', '이상', '절차', '사실', '우리나라', '요구', '구체적', '요건', '영향', '재산', '평가', '상황', '이해', '가능성', '이용', '방식', '당사자', '제공', '검토', '도입', '부담', '않는다', '견해', '제시', '독일', '생각', '손해', '제기', '위하여', '차이', '나타', '일본', '대해서는', '등을', '않는', '모두', '주식', '이후', '않은', '침해', '취득', '입장', '연구', '부과', '입법', '현재', '일반적', '위반', '처벌', '개인', '하나', '처분', '부분', '성립', '채권자', '동일', '모든', '분석', '이전', '소송', '점에서', '정도', '부동산', '지급', '규제', '사회', '피해자', '경찰', '자신', '허용', '구성', '증가', '의사',

In [18]:
new_word = []
for t in tqdm(law_word_list):
    if t not in vocab_tokens:
        new_word.append(t)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [19]:
# 기존 vocab과 겹치는 단어 확인
for t in tqdm(law_word_list):
    if t in vocab_tokens:
        print(t)

  0%|          | 0/10000 [00:00<?, ?it/s]

있다
경우
규정
이러한
문제
필요
인정
행위
관련
적용
것으로
때문
또는
하는
것이
관한
판단
발생
해당
법률
같은
계약
위한
이를
의미
회사
기업
책임
사건
제도
의무
관계
이에
결과
보호
해석
결정
대해서
권리
내용
원칙
개정
대해
국가
고려
없는
같이
기준
정보
때문에
범죄
거래
판결
이익
존재
우리
포함
이유
근거
주주
주장
있어서
개념
논의
가능
청구
통해
제한
필요가
사용
법적
목적
헌법
다음
행사
중요
위해
방법
법인
미국
대상
이상
절차
사실
우리나라
요구
영향
재산
평가
상황
이해
가능성
이용
방식
당사자
제공
검토
도입
부담
않는다
제시
독일
생각
손해
제기
위하여
차이
나타
일본
대해서는
등을
않는
모두
주식
이후
않은
침해
취득
입장
연구
부과
입법
현재
위반
처벌
개인
하나
처분
부분
성립
동일
모든
분석
이전
소송
정도
부동산
지급
규제
사회
피해자
경찰
자신
허용
구성
증가
의사
효과
일부
위험
다양
실제
달리
재판
자유
가장
이행
구분
변경
자체
현행
정부
대법원
위해서는
이사
보장
기간
확인
제정
소득
있고
해결
비용
특별
기능
활용
일정
사이
가치
수사
수도
법원
특정
집행
금지
시장
설명
지위
사람
새로
형태
유지
운영
그러
사항
양도
타당
한편
국민
없이
역시
기존
이미
상당
수행
직접
변화
확정
여러
신고
사유
회피
권한
최근
증거
반영
우선
사안
부여
성격
행정
검사
일반
마련
물론
그것
진행
함께
기관
보고
가지고
사회적
정의
보험
인하
수준
취소
이하
정책
방안
충분
확대
인식
시행
지역
사업
체결
국내
제외
유사
토지
합의
피고
당시
확보
대상이
배상
구조
수단
과정에서
업무
관리
조사
상대방
역할
있는지
의견
가능성이
활동
한계
더욱
교육
이들
다수
사실상
에서
등이
내지
전체
경제적
것인지
불구하고
결국
금액
소비자
비교
없다고
조정
표현
않았다
중국
대상으로
사례
부정
과정
단순
대부분
반면
또한
파악
인해
납부
강화
주체
자기
한국
처리
무효
신청
조치
발전
방지
목적으로
어떠
추정
합리적
어떤
개선
어렵다


어린이
책임감
보편
치매
정말
총리
차기
게이
도발
가득
득실
강원
불구
습격
공산
은퇴
머리
육군
단호
초청
카풀
도장
대치
시계
화력
잔재
유대
감옥
대사
분단
진동
진영
치부
정답
대수
무차별
트위터
대학생
거짓말
되풀이
아니냐
자녀들
청문회
컨트롤
검찰개혁
우리사회
정신병자
페이스북
자원봉사
한국전쟁
무능력
투표권
수혜
한일
농민
될수
와서
명단
벌써
부탁
중증
도쿄
취향
녹취
일대
간첩
오후
시달
착각
부류
귀국
편법
궁금
집안
낚시
수고
손자
날짜
용납
덕분
대륙
블랙
볼만
악플
고장
흑자
장면
포용
작년
망명
커버
비위
생리
피부
이사장
공산당
비정상
사모펀드
공산주의
비정규직
내려야
당선
형제
폭락
집권
사랑
목숨
진상
승리
사죄
유착
지갑
식물
내수
매력
중고
교훈
폭로
범법
신변
상주
추방
악법
전수
남발


In [20]:
print(len(law_word_list))
print('-------------')
print(len(new_word))
print(new_word)

10000
-------------
7321
['경우에는', '민법', '여부', '판례', '조세', '형법', '과세', '범위', '구체적', '요건', '견해', '일반적', '채권자', '점에서', '측면', '지방세', '효력', '조항', '채권', '원고', '조세회피', '상법', '채무자', '실질적', '관점', '다음과', '채무', '유형', '요소', '취지', '설정', '필요성', '관하여', '납세자', '피고인', '별도', '점을', '소유권', '청구권', '형벌', '문제점', '합병', '지방자치단체', '신탁', '법원은', '원칙적', '착오', '동시', '적극적', '징수', '귀속', '취하', '기본권', '등기', '기본적', '산정', '전제로', '담보', '법리', '행위자', '보아야', '세율', '선박', '법령', '체계', '상대적', '타인', '이사회', '부합', '급부', '쟁점', '취득세', '충족', '약관', '차원', '직접적', '의결권', '경영자', '세법', '반환', '소유자', '개정안', '판시', '작용', '규범', '채택', '학설', '현실적', '재산권', '배당', '구성요건', '조에서', '비중', '민법상', '하여야', '수익자', '헌법재판소', '필요성이', '관련성', '위임', '종류', '약정', '투자자', '통지', '형사소송법', '세액', '사용자', '운송', '법률상', '자회사', '법익', '공제', '주관적', '부가가치세', '규율', '배우자', '효력이', '대상판결', '조약', '보험자', '감면', '가액', '유효', '개별적', '양형', '위원회', '번째', '민사', '측정', '전통적', '종래', '분할', '과세하', '유의', '위험성', '지속적', '수집', '세목', '양자', '배분', '본질적', '증여세', '위법성', '납세', '결합', '회사법', '후자', '운송인', '결의', 

### (3) 기존 vocab에 단어 추가

In [21]:
tokenizer.add_tokens(new_word)
len(tokenizer)

37321

In [22]:
print({index:tok for tok, index in tokenizer.get_vocab().items()})

{29879: '선택이', 25099: '시위를', 18669: '터지면', 17639: '않도록', 37032: '하드웨어', 1984: '슥', 26059: '동조하는', 855: '뎄', 13266: '부대', 20850: '개천', 11907: '##하는거냐', 5863: '##옘', 25310: '공산독재', 29651: '##부름', 3378: '핶', 23821: '것이라고', 33347: '복원', 19665: '할인', 9951: '##입니까', 17562: '##스도', 22717: '조국에게', 29170: '주제를', 33876: '법리상', 9829: '뇌가', 4684: '##쩔', 1229: '룍', 4151: '##언', 7283: '##쟙', 12752: '##중국', 22150: '기레기들아', 35207: '특정성', 8803: '나이', 24170: '싸지르', 3734: '👑', 17503: '##꺼냐', 35092: '정신감정', 16366: '장하', 21531: '말이죠', 9991: '러시아', 10223: '##조리', 29780: '원칙을', 3571: '🌑', 4097: '##싱', 35494: '문언적', 18696: '어딨냐', 9780: '생산', 12889: '지지자들', 28134: '준비를', 36388: '중혼적', 32595: '신인의무', 3772: '💏', 4479: '##강', 24077: '##할꼬', 34784: '모독죄', 32805: '위헌적', 34172: '통로', 15244: '##지금', 13375: '없는거', 29328: '##할꺼다', 12621: '##어놓고', 18486: '천안', 18007: '않지', 18775: '##음에', 18991: '##짓만', 36385: '마이닝', 5566: '##잊', 35151: '논제', 25840: '욕이나', 357: '굻', 18459: '비극', 22237: '바미당', 7693: '##읨', 22634: '흔들고', 5

In [23]:
mlm_model = AutoModelForMaskedLM.from_pretrained(weight_path)
embed = mlm_model.resize_token_embeddings(len(tokenizer))
mlm_model.set_input_embeddings(embed)

mlm_model

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(37321, 768)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

In [24]:
tokenizer.save_pretrained(new_weight_path)
mlm_model.save_pretrained(new_weight_path)

In [25]:
t = tokenizer.convert_ids_to_tokens([i for i in range(tokenizer.vocab_size, len(tokenizer))])

with open(os.path.join(new_weight_path, 'vocab.txt'), 'a', encoding='utf-8') as f:
    print('\n'.join(t), file=f)
    os.remove(os.path.join(new_weight_path, 'added_tokens.json'))

### (4) Tokenizer 확인

In [26]:
txt = law_train['pre_content'][125]
txt

'채무의 대상이 되는 목적물이 조합재산에 속한다면 연대채무자인 각 조합원은 단독으로 자신의 채무를 이행할 수 없다 '

In [27]:
kc_tokenizer = AutoTokenizer.from_pretrained(weight_path)
len(kc_tokenizer)

30000

In [28]:
new_tokenizer = AutoTokenizer.from_pretrained(new_weight_path)
len(new_tokenizer)

37321

In [29]:
len(kc_tokenizer.encode_plus(txt)['input_ids'])

30

In [30]:
len(new_tokenizer.encode_plus(txt)['input_ids'])

27

In [31]:
kc_tokenizer.tokenize(txt)

['채',
 '##무',
 '##의',
 '대상이',
 '되는',
 '목적',
 '##물이',
 '조합',
 '##재산',
 '##에',
 '속',
 '##한다면',
 '연대',
 '##채',
 '##무',
 '##자인',
 '각',
 '조합',
 '##원은',
 '단독',
 '##으로',
 '자신의',
 '채',
 '##무를',
 '이행',
 '##할',
 '수',
 '없다']

In [32]:
new_tokenizer.tokenize(txt)

['채무',
 '의',
 '대',
 '상이',
 '되는',
 '목적물',
 '이',
 '조합재산',
 '에',
 '속',
 '##한다면',
 '연대채무자',
 '인',
 '각',
 '조합원',
 '은',
 '단독',
 '##으로',
 '자신의',
 '채무',
 '를',
 '이행',
 '##할',
 '수',
 '없다']

### (5) Model 확인

In [33]:
kc_model = AutoModel.from_pretrained(weight_path)
print(len(kc_model.embeddings.word_embeddings.weight))
kc_model.embeddings.word_embeddings.weight

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


30000


Parameter containing:
tensor([[ 0.0068,  0.0405,  0.0532,  ..., -0.0005, -0.0037, -0.0075],
        [-0.0118,  0.0223, -0.0034,  ..., -0.0258,  0.0345, -0.0261],
        [-0.0067, -0.0742,  0.0100,  ..., -0.0203,  0.0141,  0.0092],
        ...,
        [-0.0179, -0.0272,  0.0409,  ...,  0.0047,  0.0600, -0.0507],
        [ 0.0106, -0.0139,  0.0093,  ..., -0.0235, -0.0123,  0.0242],
        [ 0.0407,  0.0406, -0.0185,  ...,  0.0224,  0.0091,  0.0027]],
       requires_grad=True)

In [34]:
new_model = AutoModel.from_pretrained(new_weight_path)
print(len(new_model.embeddings.word_embeddings.weight))
new_model.embeddings.word_embeddings.weight

Some weights of the model checkpoint at ./fp_result/law_further_pretrained were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./fp_result/law_further_pretrained and are newly initialized: ['bert.pooler.dense.bias', '

37321


Parameter containing:
tensor([[ 0.0068,  0.0405,  0.0532,  ..., -0.0005, -0.0037, -0.0075],
        [-0.0118,  0.0223, -0.0034,  ..., -0.0258,  0.0345, -0.0261],
        [-0.0067, -0.0742,  0.0100,  ..., -0.0203,  0.0141,  0.0092],
        ...,
        [-0.0379,  0.0086, -0.0151,  ..., -0.0183,  0.0493,  0.0097],
        [ 0.0003,  0.0086, -0.0165,  ...,  0.0119, -0.0154,  0.0249],
        [-0.0010, -0.0155, -0.0002,  ...,  0.0099, -0.0315, -0.0260]],
       requires_grad=True)

In [35]:
new_model.embeddings.word_embeddings.weight[:30000,]

tensor([[ 0.0068,  0.0405,  0.0532,  ..., -0.0005, -0.0037, -0.0075],
        [-0.0118,  0.0223, -0.0034,  ..., -0.0258,  0.0345, -0.0261],
        [-0.0067, -0.0742,  0.0100,  ..., -0.0203,  0.0141,  0.0092],
        ...,
        [-0.0179, -0.0272,  0.0409,  ...,  0.0047,  0.0600, -0.0507],
        [ 0.0106, -0.0139,  0.0093,  ..., -0.0235, -0.0123,  0.0242],
        [ 0.0407,  0.0406, -0.0185,  ...,  0.0224,  0.0091,  0.0027]],
       grad_fn=<SliceBackward>)

## 6. MLM(Further Pre-training)

In [36]:
new_mlm_model = AutoModelForMaskedLM.from_pretrained(new_weight_path)

### (1) MLM 형태로 변환

In [37]:
def tokenize_function(sentence):
    text = new_tokenizer(sentence, add_special_tokens=True, max_length=max_len, padding='max_length', truncation=True)
    text['labels'] = text['input_ids'].copy()
    return text

In [38]:
law_train['token'] = law_train['pre_content'].apply(tokenize_function)
law_train

,subdomain,pre_content,content,label,token
75220,판례,인수합병을 추진하는 입장에서 피고인에게 회사일을 거론할 수는 없었다는 정의 법정 진...,인수합병을 추진하는 입장에서 피고인에게 회사일을 거론할 수는 없었다는 정의 법정 진...,3,"[input_ids, token_type_ids, attention_mask, la..."
48955,세법,이는 차명거래를 한 데 대한 제재의 성격을 갖고 있다고 볼 수밖에 없다,이는 차명거래를 한 데 대한 제재의 성격을 갖고 있다고 볼 수밖에 없다.,2,"[input_ids, token_type_ids, attention_mask, la..."
44966,세법,행정 담당 공무원들은 년 담당하다가 자리를 옮기니 전문성을 갖출 시간이 부족하다,행정 담당 공무원들은 1∼2년 담당하다가 자리를 옮기니 전문성을 갖출 시간이 부족하다.,2,"[input_ids, token_type_ids, attention_mask, la..."
13568,민법,우리 민법은 이러한 일본민법의 태도를 따른 것이다,우리 민법은 이러한 일본민법의 태도를 따른 것이다.,0,"[input_ids, token_type_ids, attention_mask, la..."
92727,형법,채굴을 통해서 생성된 블록이 블록체인에 결합되기 위해서는 일정한 요건을 충족해야 한다,채굴을 통해서 생성된 블록이 블록체인에 결합되기 위해서는 일정한 요건을 충족해야 한다.,4,"[input_ids, token_type_ids, attention_mask, la..."
...,...,...,...,...,...
6265,민법,머신 러닝 기술을 활용하면 라는 미지의 입력 내용에 대해서도 컴퓨터는 이렇게 하면 ...,머신 러닝 기술을 활용하면 Z라는 미지의 입력 내용에 대해서도 컴퓨터는 이렇게 하면...,0,"[input_ids, token_type_ids, attention_mask, la..."
54886,세법,이에 따르면 시스템에 대한 사용자의 일반적 만족도는 동 시스템 구축 이전 자치단체별...,"이에 따르면, 시스템에 대한 사용자의 일반적 만족도는 동 시스템 구축 이전 자치단체...",2,"[input_ids, token_type_ids, attention_mask, la..."
76820,판례,근본적으로는 국내 해사 중재 활성화를 위한 기본 토대가 갖춰져 있지 않다는 점을 들...,"근본적으로는, 국내 해사 중재 활성화를 위한 기본 토대가 갖춰져 있지 않다는 점을 ...",3,"[input_ids, token_type_ids, attention_mask, la..."
860,민법,또한 는 증권거래세를 부과함에 따라 과세정보의 외부효과로 인하여 자원의 낭비를 감소...,"또한, AAA는 증권거래세를 부과함에 따라 과세정보의 외부효과로 인하여 자원의 낭비...",0,"[input_ids, token_type_ids, attention_mask, la..."


In [39]:
law_val['token'] = law_val['pre_content'].apply(tokenize_function)
law_val

,subdomain,pre_content,content,label,token
75721,판례,마지막으로 년에는 그동안 부진했던 증권 관련 집단소송 중 여러 건이 진행되었다,마지막으로 2018년에는 그동안 부진했던 증권 관련 집단소송 중 여러 건이 진행되었다.,3,"[input_ids, token_type_ids, attention_mask, la..."
80184,형법,사법 해석은 법률의 규정이나 입법 취지를 벗어나 해석할 수 없고 범죄 자산 몰수 특...,"사법 해석은 법률의 규정이나 입법 취지를 벗어나 해석할 수 없고, 범죄 자산 몰수 ...",4,"[input_ids, token_type_ids, attention_mask, la..."
19864,민법,이 사건 소가 전소에서 확정된 법률관계와 정반대의 모순되는 사항을 소송물로 하는 것...,이 사건 소가 전소에서 확정된 법률관계와 정반대의 모순되는 사항을 소송물로 하는 것...,0,"[input_ids, token_type_ids, attention_mask, la..."
76699,판례,법적 관점에서도 태아가 불법행위로 인한 손해배상청구권을 가진다,법적 관점에서도 태아가 불법행위로 인한 손해배상청구권을 가진다.,3,"[input_ids, token_type_ids, attention_mask, la..."
92991,형법,즉 생명권은 사람의 생존본능과 존재목적 고유한 존재가치에 바탕을 두고 있으므로 이는...,"즉, 생명권은 사람의 생존본능과 존재목적 고유한 존재가치에 바탕을 두고 있으므로 이...",4,"[input_ids, token_type_ids, attention_mask, la..."
...,...,...,...,...,...
32595,법률연구,대량 실업 사태와 임금 저하 현상이 속출하였고 년 노동법 체계에 대한 재검토가 불가...,"대량 실업 사태와 임금 저하 현상이 속출하였고, 1997년 노동법 체계에 대한 재검...",1,"[input_ids, token_type_ids, attention_mask, la..."
29313,법률연구,사건에서 영국 는 새로운 기술에 기초한 디자인의 경우 그러한 새로운 기술이 디자이...,"Dyson Ltd v Vax Ltd 사건에서, 영국 High Court는 새로운 기...",1,"[input_ids, token_type_ids, attention_mask, la..."
37862,법률연구,자유법론의 주장은 제정법과 법의 무흠결성이란 도그마 형식논리 법학에서의 의지 작용의...,"자유법론의 주장은 제정법과 법의 무흠결성이란 도그마, 형식논리, 법학에서의 의지 작...",1,"[input_ids, token_type_ids, attention_mask, la..."
53421,세법,는 구성요소를 분리하는 것이 어렵기 때문에 이익조정 및 회계기준과 세법의 기계적인...,BTD는 구성요소를 분리하는 것이 어렵기 때문에 이익조정 및 회계기준과 세법의 기계...,2,"[input_ids, token_type_ids, attention_mask, la..."


In [40]:
law_train['token'].iloc[0]

{'input_ids': [2, 34070, 2424, 12166, 7966, 14295, 30034, 15389, 8963, 9858, 17654, 4082, 15358, 12629, 4008, 8209, 19711, 14340, 4017, 12665, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids'

In [41]:
data_collator = DataCollatorForLanguageModeling(tokenizer=new_tokenizer, mlm_probability=0.15)

In [42]:
mlm_train = law_train['token'].copy()
mlm_train.reset_index(drop=True, inplace=True)

mlm_val = law_val['token'].copy()
mlm_val.reset_index(drop=True, inplace=True)

In [43]:
mlm_train[100]

{'input_ids': [2, 21380, 8966, 8229, 10041, 13867, 12810, 4105, 9186, 8158, 16817, 4102, 7975, 7968, 10794, 13804, 4042, 18561, 9878, 7966, 12710, 35201, 13256, 4072, 4042, 11794, 30023, 2454, 18861, 8094, 26503, 8294, 8060, 4047, 30782, 2451, 903, 8556, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### (2) train

In [44]:
training_args = TrainingArguments(output_dir=new_weight_path,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  overwrite_output_dir=True,
                                  num_train_epochs=epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  seed=seed)

trainer = Trainer(model=new_mlm_model,
                  args=training_args,
                  data_collator=data_collator,
                  train_dataset=mlm_train,
                  eval_dataset=mlm_val)

In [45]:
torch.cuda.empty_cache()

In [46]:
trainer.train()

D:\Anaconda3\envs\minah\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 80000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 50000
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.762300,3.620619
2,3.402700,3.358095
3,3.196000,3.183851
4,3.043500,3.067059
5,2.903500,2.971504
6,2.783000,2.879655
7,2.674200,2.812704
8,2.581900,2.777443
9,2.503700,2.732114
10,2.440000,2.708471


***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ./fp_result/law_further_pretrained\checkpoint-2500
Configuration saved in ./fp_result/law_further_pretrained\checkpoint-2500\config.json
Model weights saved in ./fp_result/law_further_pretrained\checkpoint-2500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ./fp_result/law_further_pretrained\checkpoint-5000
Configuration saved in ./fp_result/law_further_pretrained\checkpoint-5000\config.json
Model weights saved in ./fp_result/law_further_pretrained\checkpoint-5000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ./fp_result/law_further_pretrained\checkpoint-7500
Configuration saved in ./fp_result/law_further_pretrained\checkpoint-7500\config.json
Model weights saved in ./fp_result/law_further_pretrained\checkpoint-7500\pytorch_model.bin
***** Running Ev

TrainOutput(global_step=50000, training_loss=2.5754007043457032, metrics={'train_runtime': 38654.0543, 'train_samples_per_second': 41.393, 'train_steps_per_second': 1.294, 'total_flos': 2.4677409312e+17, 'train_loss': 2.5754007043457032, 'epoch': 20.0})

In [47]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32


{'eval_loss': 2.5084939002990723,
 'eval_runtime': 146.0947,
 'eval_samples_per_second': 136.898,
 'eval_steps_per_second': 4.278,
 'epoch': 20.0}

In [50]:
mlm_train.to_pickle('./data/law_mlm_train.pickle')
mlm_val.to_pickle('./data/law_mlm_val.pickle')